In [1]:
import pandas as pd

## read file
def read_file(year,month,day):
    data = []
    if len(str(month)) == 1:
        month_ = '0'+str(month)
    else:
        month_ = str(month)
    if len(str(day)) == 1:
        day_ = '0'+str(day)
    else:
        day_ = str(day)
    datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/CN_Futures_'+str(year)+'.'+str(month_)+'.'+str(day_)+'.csv'
    data = pd.read_csv(datapath)
    data_CN14 = data[data.Series == data['Series'][0]]
    data = data_CN14
    return data

In [18]:
data = read_file(2014,1,28)

In [19]:
import time 
import csv
with open('order_book_2014_1_28.csv', 'wb') as csvfile:
    f = csv.writer(csvfile)
def insert(order_book_data,data_to_insert,ob_position):
    top = order_book_data[0:ob_position]
    bottom = order_book_data[ob_position:]
    return pd.concat((top,data_to_insert,bottom)).reset_index(drop = True)
def draw_out(order_book_data,ob_position):
    top = order_book_data[0:ob_position]
    bottom = order_book_data[ob_position+1:]
    return pd.concat((top,bottom)).reset_index(drop = True)
data['QuantityDifference_'] = data['QuantityDifference']
data_ask = data[(data.BidOrAsk == 'A')].reset_index(drop=True)
data_bid = data[(data.BidOrAsk == 'B')].reset_index(drop=True)
start  =  time.time()
order_book_bid = []
order_book_ask = []
timestamp = data.TimeStamp[0]
temp_ask = 0
temp_bid = 0
bid = []
ask = []
timestamp_ = []
x1 = data[(data.BidOrAsk == 'A')].TimeStamp.unique()
x2 = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
y = data[data['TimeStamp'].str.contains(timestamp)]
bid.append(y[(y.BidOrAsk == 'B')][["Price","OrderNumber","QuantityDifference","QuantityDifference_"]]) # bid
ask.append(y[(y.BidOrAsk == 'A')][["Price","OrderNumber","QuantityDifference","QuantityDifference_"]]) # ask
a = bid[0].sort(['Price'],ascending = [False])
b = ask[0].sort(['Price'],ascending = [True])    
order_book_bid.append(a[a.QuantityDifference != 0].reset_index(drop = True))
order_book_ask.append(b[b.QuantityDifference != 0].reset_index(drop = True))
order_book_bid.append(0)
order_book_ask.append(0)
order_book_bid_sum = order_book_bid[0][['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False,sort = False).sum()
order_book_ask_sum = order_book_ask[0][['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()
order_book_bid_ask = pd.concat([order_book_bid_sum[['Price','QuantityDifference']],order_book_ask_sum[['Price','QuantityDifference']]],axis = 1)

with open('order_book_2014_1_28.csv','a') as f:
    order_book = csv.writer(f)
    order_book.writerow(["Bid","Bid_Quantity","Ask","Ask_Quantity"])
    order_book.writerow(["TimeStamp",timestamp])
    order_book = csv.writer(f,delimiter=',')
    for i in range(0,max(len(order_book_ask[0]),len(order_book_bid[0]))):
        order_book.writerow(order_book_bid_ask[i:i+1].values.tolist()[0])
        

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:30: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:31: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [20]:
def order_book_to_csv(order_book_bid,order_book_ask,data,i):
    order_book_bid_sum = order_book_bid[['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False,sort = False).sum()
    order_book_ask_sum = order_book_ask[['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()
    order_book_bid_ask = pd.concat([order_book_bid_sum[['Price','QuantityDifference']],order_book_ask_sum[['Price','QuantityDifference']]],axis = 1)    
    #print order_book_bid_ask
    with open('order_book_2014_1_28.csv','a') as f:
        order_book = csv.writer(f)
        order_book.writerow(["TimeStamp",data.TimeStamp[i-1:i].iloc[0]])
        order_book = csv.writer(f,delimiter=',')
        for i in range(0,len(order_book_bid_ask)):
            order_book.writerow(order_book_bid_ask[i:i+1].values.tolist()[0])
    return order_book_bid_sum,order_book_ask_sum

In [21]:
for i in range(len(y),5000,1):
    print i#,temp_bid,temp_ask
    time_second = int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                  int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                  int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
    if time_second == 32400:
        order_book_bid[temp_bid] = order_book_bid[temp_bid].sort(['Price'],ascending = [False]).reset_index(drop = True)
        order_book_ask[temp_ask] = order_book_ask[temp_ask].sort(['Price'],ascending = [True]).reset_index(drop = True)
        pass 
    if data.BidOrAsk[i] == 'A':
        data_ask_Quantity = data.BestQuantity[i]
        if int(data[['QuantityDifference']][i:i+1].values) > 0 :
            if order_book_bid[temp_bid].Price[0] >= data[i:i+1].Price.iloc[0] and time_second < 32400:
                for k in range(0,len(order_book_bid[temp_bid])):
                    diff = order_book_bid[temp_bid].QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]
                    if order_book_bid[temp_bid].Price[k] >= data[i:i+1].Price.iloc[0] and diff >= 0:
                        order_book_bid[temp_bid].QuantityDifference_[k] = diff
                        data[i:i+1].QuantityDifference_.iloc[0] = 0
                        break
                    elif order_book_bid[temp_bid].Price[k] >= data[i:i+1].Price.iloc[0] and diff < 0:
                        order_book_bid[temp_bid].QuantityDifference_[k] = 0
                        data[i:i+1].QuantityDifference_.iloc[0] = -diff
                        pass
                    else:
                        break
            if data.TimeStamp[i] == x1[temp_ask]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                length_order_book_zero = len(order_book_ask[temp_ask][(order_book_ask[temp_ask].Price <= data['Price'][i:i+1].iloc[0]) & (order_book_ask[temp_ask].QuantityDifference_ == 0)])
                if time_second > 32400:
                    if position_ == 0 and len(order_book_ask[0]) > 1:
                        if order_book_ask[temp_ask][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:  
                            print 'Some error1(Ask & Q>0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< (len(order_book_ask[temp_ask])-1):
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:#or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error1(Ask & Q>0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'   
                            pass
                    elif position_ == len(order_book_ask[temp_ask])-1:
                        if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error1(Ask & Q>0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                
                            pass
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True' 
                        pass
                else:
                    print 'before open market'
            elif data.TimeStamp[i] != x1[temp_ask]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                if temp_ask == 0:
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = order_book_ask[temp_ask - 1]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)                  
                    if time_second > 32400:
                        if position_ == 0 and len(order_book_ask[0]) > 1:
                            if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 

                                print 'Some error2(Ask & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                break
                            else:
                                print 'position and Best Price is True'                     
                                pass
                        elif 0 < position_< len(order_book_ask[temp_ask])-1:
                            if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ -  1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 

                                print 'Some error2(Ask & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                                break
                            else:
                                print 'position and Best Price is True'  
                                pass
                        elif position_ == len(order_book_ask[temp_ask])-1:   
                            if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 

                                print 'Some error2(Ask & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                break
                            else:
                                print 'position and Best Price is True'
                                pass
                        elif position_ == 0 and len(order_book_ask[0]) == 1:
                            print 'position and Best Price is True'
                            pass
                    else:
                        print 'before open market'
                else:
                    order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)
                    if time_second > 32400:
                        if data[i-1:i].BidOrAsk.iloc[0] == 'A':
                            if order_book_ask_sum[0:1].values.tolist()[0][1] == data[i-1:i].BestQuantity.iloc[0]:
                                print 'Best ask quantity is ture'
                                pass
                            else:
                                print 'Best ask quantity is false'
                                break 
                        else:
                            j = i - 1
                            while j >= 1:
                                print j
                                print data[j-1:j].BidOrAsk.iloc[0]
                                if data[j-1:j].BidOrAsk.iloc[0] == 'A':
                                    if order_book_ask_sum[0:1].values.tolist()[0][1] == data[j-1:j].BestQuantity.iloc[0]:
                                        print 'Best ask quantity is ture'
                                        break
                                else:
                                    j = j - 1
                                    pass   
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    order_book_ask.append(order_book_ask[temp_ask])
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)#int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                    if time_second > 32400:
                        if position_ == 0:
                            if order_book_ask[temp_ask][position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 

                                print 'Some error3(Ask & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                break
                            else:
                                print 'position and Best Price is True'                     
                                pass
                        elif 0 < position_< len(order_book_ask[temp_ask])-1:
                            if order_book_ask[temp_ask][position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 

                                print 'Some error3(Ask & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                                break
                            else:
                                print 'position and Best Price is True'  
                                pass
                        elif position_ == len(order_book_ask[temp_ask])-1:    
                            if order_book_ask[temp_ask][position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 

                                print 'Some error3(Ask & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                break
                            else:
                                print 'position and Best Price is True'                        
                                pass
                        elif position_ == 0 and len(order_book_ask[0]) == 1:
                            print 'position and Best Price is True'
                            pass
                    else:
                        print 'before open market'
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    
            if data.TimeStamp[i] == x1[temp_ask]: 
                print 'timestamp = %s'%(data.TimeStamp[i])
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                position_ = order_book_ask[temp_ask][order_book_ask[temp_ask].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                if time_second > 32400:
                    if position_ == 0 and len(order_book_ask[0]) > 1:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 

                            print 'Some error4(Ask & Q<0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass       
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 

                            print 'Some error4(Ask & Q<0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_ask[temp_ask])-1:       
                        if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 

                            print 'Some error4(Ask & Q<0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])   
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            pass                        
                        else:
                            print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        pass
                else:
                    print 'before open market'                   
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
            elif data.TimeStamp[i] != x1[temp_ask]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)
                if time_second > 32400:
                    if data[i-1:i].BidOrAsk.iloc[0] == 'A':
                        if order_book_ask_sum[0:1].values.tolist()[0][1] == data[i-1:i].BestQuantity.iloc[0]:
                            print 'Best ask quantity is ture'
                            pass
                        else:
                            print 'Best ask quantity is false'
                            break 
                    else:
                        j = i - 1
                        while j >= 1:
                            print j
                            print data[j-1:j].BidOrAsk.iloc[0]
                            if data[j-1:j].BidOrAsk.iloc[0] == 'A':
                                if order_book_ask_sum[0:1].values.tolist()[0][1] == data[j-1:j].BestQuantity.iloc[0]:
                                    print 'Best ask quantity is ture'
                                    break
                            else:
                                j = j - 1
                                pass
                order_book_ask.append(order_book_ask[temp_ask])
                order_number_ =  data['OrderNumber'][i : i + 1].iloc[0]
                position_ = order_book_ask[temp_ask][order_book_ask[temp_ask].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_ask = temp_ask + 1
                if time_second > 32400:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 

                            print 'Some error5(Ask & Q<0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 

                            print 'Some error5(Ask & Q<0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_ask[temp_ask])-1:

                        if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 

                            print 'Some error5(Ask & Q<0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            pass                       
                        else:
                            print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        pass      
                else:
                    print 'before open market'
                    
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                    
    elif data.BidOrAsk[i] == 'B':
        data_bid_Quantity = data.BestQuantity[i]
        if int(data[['QuantityDifference']][i:i+1].values) > 0: 
            if order_book_ask[temp_ask].Price[0] <= data[i:i+1].Price.iloc[0] and time_second < 32400:
                for k in range(0,len(order_book_ask[temp_ask])):
                    diff = order_book_ask[temp_ask].QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]
                    if order_book_ask[temp_ask].Price[k] <= data[i:i+1].Price.iloc[0] and diff >= 0:
                        order_book_ask[temp_ask].QuantityDifference_[k] = diff
                        data[i:i+1].QuantityDifference_.iloc[0] = 0
                        break
                    elif order_book_ask[temp_ask].Price[k] <= data[i:i+1].Price.iloc[0] and diff < 0:
                        order_book_ask[temp_ask].QuantityDifference_[k] = 0
                        data[i:i+1].QuantityDifference_.iloc[0] = - diff
                        pass
                    else:
                        break
            if data.TimeStamp[i] == x2[temp_bid]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                length_order_book_zero = len(order_book_bid[temp_bid][(order_book_bid[temp_bid].Price >= data['Price'][i:i+1].iloc[0]) & (order_book_bid[temp_bid].QuantityDifference_ == 0)])
                if time_second > 32400:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:                
                            print 'Some error1(Bid & Q>0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass     
                    elif 0 < position_< len(order_book_bid[temp_bid])-1: 
                        if order_book_bid[temp_bid][position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error1(Bid & Q>0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break    
                        else:
                            print 'position and Best Price is True'   
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1 and len(order_book_bid[0]) > 1:      
                        if order_book_bid[temp_bid][position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                            print 'Some error1(Bid & Q>0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                
                            pass
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        pass 
                else:
                    print 'before open market'
                    pass
            elif data.TimeStamp[i] != x2[temp_bid]:
                print temp_bid
                print 'timestamp = %s'%(data.TimeStamp[i])
                if temp_bid == 0:
                    best_price = data[i:(i+1)]['BestPrice']             
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = order_book_bid[temp_bid - 1]
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)                 
                    if time_second > 32400:
                        if position_ == 0  and len(order_book_bid[0]) > 1:
                            if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                print 'Some error2(Bid & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                                break
                            else:
                                print 'position and Best Price is True'                     
                                pass       
                        elif 0 < position_< len(order_book_bid[temp_bid])-1:
                            if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                                print 'Some error2(Bid & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                print 'position and Best Price is True'  
                                pass
                        elif position_ == len(order_book_bid[temp_bid])-1:    
                            if order_book_bid[temp_bid][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                print 'Some error2(Bid & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])                                        
                                pass
                            else:
                                print 'position and Best Price is True'
                                pass
                        elif position_ == 0 and len(order_book_bid[0]) == 1:
                                print 'position and Best Price is True'
                                pass
                    else:
                        print 'before open market'
                        pass
                else:
                    if time_second > 32400:
                        order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)                    
                        if data[i-1:i].BidOrAsk.iloc[0] == 'B':
                            if order_book_bid_sum[0:1].values.tolist()[0][1] == data[i-1:i].BestQuantity.iloc[0]:
                                print 'Best bid quantity is ture'
                                pass
                            else:
                                print 'Best bid quantity is false'
                                break 
                        else:
                            j = i - 1
                            while j >= 1:
                                print j
                                print data[j-1:j].BidOrAsk.iloc[0]
                                if data[j-1:j].BidOrAsk.iloc[0] == 'B':
                                    if order_book_bid_sum[0:1].values.tolist()[0][1] == data[j-1:j].BestQuantity.iloc[0]:
                                        print 'Best bid quantity is ture'
                                        break
                                    else:
                                        print 'Best bid quantity is false'
                                        break
                                else:
                                    j = j - 1
                                    pass
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    order_book_bid.append(order_book_bid[temp_bid])
                    temp_bid = temp_bid + 1 
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)   
                    if time_second > 32400:
                        if position_ == 0 and len(order_book_bid[0]) > 1:
                            if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                print 'Some error3(Bid & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                break
                            else:
                                print 'position and Best Price is True'                     
                                pass
                        elif 0 < position_< len(order_book_bid[temp_bid])-1:
                            if order_book_bid[temp_bid][position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                                print 'Some error3(Bid & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                print 'position and Best Price is True'  
                                pass
                        elif position_ == len(order_book_bid[temp_bid])-1:
                            if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                print 'Some error3(Bid & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                break
                            else:
                                print 'position and Best Price is True'                        
                                pass
                        elif position_ == 0 and len(order_book_bid[0]) == 1:
                            print 'position and Best Price is True'
                            pass
                    else:
                        print 'before open market'
                        pass
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    
            if data.TimeStamp[i] == x2[temp_bid]: 
                print 'timestamp = %s'%(data.TimeStamp[i])
                order_number_ =  data['OrderNumber'][i : i + 1].iloc[0]
                position_ = order_book_bid[temp_bid][order_book_bid[temp_bid].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                if time_second > 32400:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            print 'Some error4(Bid & Q<0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass               
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            print 'Some error4(Bid & Q<0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                            print 'Some error4(Bid & Q<0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            pass
                        else:
                            print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        pass
                else:
                    print 'before open market'
                    pass
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]                
            elif data.TimeStamp[i] != x2[temp_bid]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                if time_second > 32400:
                    order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)
                    if data[i-1:i].BidOrAsk.iloc[0] == 'B':
                        if order_book_bid_sum[0:1].values.tolist()[0][1] == data[i-1:i].BestQuantity.iloc[0]:
                            print 'Best bid quantity is ture'
                            pass
                        else:
                            print 'Best bid quantity is false'
                            break 
                    else:
                        j = i - 1
                        while j >= 1:
                            print j
                            print data[j-1:j].BidOrAsk.iloc[0]
                            if data[j-1:j].BidOrAsk.iloc[0] == 'B':
                                if order_book_bid_sum[0:1].values.tolist()[0][1] == data[j-1:j].BestQuantity.iloc[0]:
                                    print 'Best bid quantity is ture'
                                    break
                                else:
                                    print 'Best bid quantity is false'
                                    break
                            else:
                                j = j - 1
                                pass 
                order_book_bid.append(order_book_bid[temp_bid])
                order_number_ =  data['OrderNumber'][i:i+1].iloc[0]
                position_ = order_book_bid[temp_bid][order_book_bid[temp_bid].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_bid = temp_bid + 1
                if time_second > 32400:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            print 'Some error5(Bid & Q<0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            print 'Some error5(Bid & Q<0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1: 
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                            print 'Some error5(Bid & Q<0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            pass
                        else:
                            print 'position and Best Price is True'                        
                            pass    
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        pass  
                else:
                    print 'before open market'
                    pass
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
end = time.time()            
print "Total time = %f"%(end - start)

2
0
timestamp = 2014-01-28D08:45:01.501087400
before open market
3
1
timestamp = 2014-01-28D08:45:01.584087400
before open market
4
2
timestamp = 2014-01-28D08:45:02.060075400
before open market
5
3
timestamp = 2014-01-28D08:45:02.062075400
before open market
6
4
timestamp = 2014-01-28D08:45:02.209075400
before open market
7
5
timestamp = 2014-01-28D08:45:02.374075400
before open market
8
timestamp = 2014-01-28D08:45:27.597775400
before open market
9
timestamp = 2014-01-28D08:45:32.346715400
before open market
10
timestamp = 2014-01-28D08:45:44.637571400
before open market
11
6
timestamp = 2014-01-28D08:46:00.063379400
before open market
12
7
timestamp = 2014-01-28D08:46:00.064379400
before open market
13
8
timestamp = 2014-01-28D08:46:00.067379400
before open market
14
9
timestamp = 2014-01-28D08:46:00.069379400
before open market
15
timestamp = 2014-01-28D08:46:00.070379400
before open market
16
10
timestamp = 2014-01-28D08:46:00.072379400
before open market
17
timestamp = 2014-01-28

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:269: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


before open market
42
timestamp = 2014-01-28D08:46:52.182755400
before open market
43
timestamp = 2014-01-28D08:46:55.028724400
before open market
44
timestamp = 2014-01-28D08:47:02.199635400
before open market
45
24
timestamp = 2014-01-28D08:47:02.730635400
before open market
46
timestamp = 2014-01-28D08:47:19.953431400
before open market
47
timestamp = 2014-01-28D08:47:30.373299400
before open market
48
25
timestamp = 2014-01-28D08:47:35.638239400
before open market
49
26
timestamp = 2014-01-28D08:47:35.675239400
before open market
50
27
timestamp = 2014-01-28D08:47:36.264227400
before open market
51
28
timestamp = 2014-01-28D08:47:42.056167000
before open market
52
timestamp = 2014-01-28D08:47:52.491035400
before open market
53
timestamp = 2014-01-28D08:48:03.749903400
before open market
54
29
timestamp = 2014-01-28D08:48:12.009807400
before open market
55
30
timestamp = 2014-01-28D08:48:15.511759400
before open market
56
timestamp = 2014-01-28D08:48:17.012747400
before open market


/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:272: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:273: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


timestamp = 2014-01-28D08:50:59.805791000
before open market
70
41
timestamp = 2014-01-28D08:51:29.802431000
before open market
71
42
timestamp = 2014-01-28D08:51:30.007431000
before open market
72
43
timestamp = 2014-01-28D08:51:47.210227000
before open market
73
timestamp = 2014-01-28D08:51:57.317107000
before open market
74
timestamp = 2014-01-28D08:51:59.639075900
before open market
75
timestamp = 2014-01-28D08:52:00.941059000
before open market
76
44
timestamp = 2014-01-28D08:52:03.867023000
before open market
77
timestamp = 2014-01-28D08:52:06.372999000
before open market
78
timestamp = 2014-01-28D08:52:10.411951000
before open market
79
45
timestamp = 2014-01-28D08:52:46.877507000
before open market
80
46
timestamp = 2014-01-28D08:52:47.011507000
before open market
81
47
timestamp = 2014-01-28D08:52:47.916495000
before open market
82
48
timestamp = 2014-01-28D08:53:38.799891500
before open market
83
49
timestamp = 2014-01-28D08:54:15.311451000
before open market
84
50
timestamp 

...

111
timestamp = 2014-01-28D08:56:04.843143000
before open market
112
timestamp = 2014-01-28D08:56:04.843143000
before open market
113
67
timestamp = 2014-01-28D08:56:05.073141200
before open market
114
timestamp = 2014-01-28D08:56:05.114137100
before open market
115
timestamp = 2014-01-28D08:56:05.155133000
before open market
116
timestamp = 2014-01-28D08:56:05.155133000
before open market
117
70
timestamp = 2014-01-28D08:56:05.378131000
before open market
118
timestamp = 2014-01-28D08:56:05.418131000
before open market
119
timestamp = 2014-01-28D08:56:05.458131000
before open market
120
timestamp = 2014-01-28D08:56:05.458131000
before open market
121
73
timestamp = 2014-01-28D08:56:05.698131000
before open market
122
timestamp = 2014-01-28D08:56:05.738131000
before open market
123
timestamp = 2014-01-28D08:56:05.778131000
before open market
124
timestamp = 2014-01-28D08:56:05.778131000
before open market
125
76
timestamp = 2014-01-28D08:56:06.027131000
before open market
126
timestamp

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:192: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


254
timestamp = 2014-01-28D09:00:00.054323000
before open market
255
135
timestamp = 2014-01-28D09:00:00.122323000
before open market
256
136
timestamp = 2014-01-28D09:00:00.123323000
before open market
257
timestamp = 2014-01-28D09:00:00.123323000
before open market
258
timestamp = 2014-01-28D09:00:00.123323000
before open market
259
timestamp = 2014-01-28D09:00:00.123323000
before open market
260
timestamp = 2014-01-28D09:00:00.123323000
before open market
261
137
timestamp = 2014-01-28D09:00:00.124323000
before open market
262
timestamp = 2014-01-28D09:00:00.124323000
before open market
263
timestamp = 2014-01-28D09:00:00.124323000
before open market
264
timestamp = 2014-01-28D09:00:00.126323000
before open market
265
timestamp = 2014-01-28D09:00:00.126323000
before open market
266
timestamp = 2014-01-28D09:00:00.126323000
before open market
267
timestamp = 2014-01-28D09:00:00.126323000
before open market
268
138
timestamp = 2014-01-28D09:00:00.128323000
before open market
269
times

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:444: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


position and Best Price is True
740
timestamp = 2014-01-28D09:01:27.088279000
position and Best Price is True
741
timestamp = 2014-01-28D09:01:27.088279000
position and Best Price is True
742
timestamp = 2014-01-28D09:01:27.089279000
Best bid quantity is ture
position and Best Price is True
743
timestamp = 2014-01-28D09:01:27.089279000
position and Best Price is True
744
timestamp = 2014-01-28D09:01:27.089279000
position and Best Price is True
745
timestamp = 2014-01-28D09:01:27.090279000
Best bid quantity is ture
position and Best Price is True
746
timestamp = 2014-01-28D09:01:27.090279000
position and Best Price is True
747
timestamp = 2014-01-28D09:01:27.090279000
746
B
745
B
744
B
743
B
742
B
741
B
740
B
739
B
738
A
Best ask quantity is ture
position and Best Price is True
748
timestamp = 2014-01-28D09:01:27.090279000
position and Best Price is True
749
timestamp = 2014-01-28D09:01:27.091279000
748
A
747
B
Best bid quantity is ture
position and Best Price is True
750
timestamp = 20

In [22]:
order_book_ask_sum,order_book_ask_sum[0:1].values.tolist()[0][1],data[j-1:j].BestQuantity.iloc[0]

(     Price  QuantityDifference
 0   671000                   8
 1   671500                  35
 2   672000                  65
 3   672500                  91
 4   673000                  37
 5   673500                  70
 6   674000                 135
 7   674500                  90
 8   675000                  77
 9   675500                  79
 10  676000                 135
 11  676500                 135
 12  677000                 397
 13  677500                   1
 14  678000                   2
 15  678500                   1
 16  679000                   1
 17  679500                   2
 18  680000                  34
 19  680500                 300
 20  684000                 400
 21  686000                   1
 22  687500                 505
 23  688000                   2
 24  688500                   9
 25  689500                   2
 26  690000                   6
 27  691000                  20
 28  693000                  10
 29  694000                   2
 30  697

In [23]:
order_book_bid_sum,order_book_bid_sum[0:1].values.tolist()[0][1],data[j-1:j].BestQuantity.iloc[0]

(     Price  QuantityDifference
 0   670500                  27
 1   670000                  60
 2   669500                  81
 3   669000                  31
 4   668500                  73
 5   668000                 230
 6   667500                  94
 7   667000                  95
 8   666500                 241
 9   666000                  70
 10  665500                 146
 11  665000                 246
 12  664500                 206
 13  664000                 268
 14  663500                   6
 15  663000                   2
 16  662500                 205
 17  662000                  51
 18  661500                   1
 19  661000                   1
 20  660500                   1
 21  660000                   1
 22  659000                 300
 23  658000                   1
 24  657000                  10
 25  656500                 400
 26  653500                   5
 27  653000                 507
 28  652500                   1
 29  647000                   1
 30  641

In [24]:
data[i:i+1]

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity,QuantityDifference_
1476,CNF14,1477,2014-01-28D09:07:27.073971000,DCE7EA019BF37F33,1,670500,-6,T,B,670500,21,-6


In [27]:
data[1470:1478]

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity,QuantityDifference_
1470,CNF14,1471,2014-01-28D09:07:12.833139000,72B26E11D24FB12F,9,670500,1,,B,670500,50,1
1471,CNF14,1472,2014-01-28D09:07:19.811055000,8B9D6D7216BCFB9D,13,671500,30,,A,671000,8,30
1472,CNF14,1473,2014-01-28D09:07:26.774971000,F833EE59EE38CF78,10,670500,1,,B,670500,51,1
1473,CNF14,1474,2014-01-28D09:07:27.072971000,72FBE601D47CB43C,1,670500,-6,T,B,670500,21,-6
1474,CNF14,1475,2014-01-28D09:07:27.072971000,FAADEE51361D19DD,1,670500,-15,T,B,670500,21,-15
1475,CNF14,1476,2014-01-28D09:07:27.072971000,FCE44A1D80D664D6,1,670500,-3,T,B,670500,21,-3
1476,CNF14,1477,2014-01-28D09:07:27.073971000,DCE7EA019BF37F33,1,670500,-6,T,B,670500,21,-6
1477,CNF14,1478,2014-01-28D09:07:27.076971000,542AE251338B16AB,20,669500,-6,,B,670500,21,-6


In [37]:
order_book_bid_sum[0:1].values.tolist()[0][1] == data[i-1:i].BestQuantity.iloc[0]

False

In [38]:
data[300:301]

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity,QuantityDifference_
300,CNG14,301,2014-01-30D08:54:03.575710600,D4EAE24552613141,8,674000,1,,B,674000,10,1


In [32]:
pd.read_table('order_book.csv',sep=',')#[0:50]

,Bid,Bid_Quantity,Ask,Ask_Quantity
0,TimeStamp,2014-01-30D04:27:08.581903900,NaN,NaN
1,665000.0,150.0,682000.0,1.0
2,663500.0,1.0,683500.0,1.0
3,663000.0,10.0,689000.0,100.0
4,660000.0,100.0,691000.0,10.0
5,NaN,NaN,699000.0,10.0
6,NaN,NaN,700000.0,1.0
7,NaN,NaN,717500.0,10.0
8,TimeStamp,2014-01-30D08:45:00.361550600,NaN,NaN
9,668000.0,1.0,682000.0,1.0


In [45]:
data[100:120]

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity,QuantityDifference_
100,CNG14,101,2014-01-30D08:49:34.924201600,87564D661B3BFD5C,4,675000,2,,A,675000,153,2
101,CNG14,102,2014-01-30D08:49:39.903136600,5821EE451C2AFF4B,41,631500,20,,B,675000,10,20
102,CNG14,103,2014-01-30D08:49:41.189110600,FAEAC645AD358C35,5,675000,6,,A,675000,159,6
103,CNG14,104,2014-01-30D08:49:53.764954600,F6296A45687949B9,2,675000,1,,A,675000,160,1
104,CNG14,105,2014-01-30D08:50:12.905707600,8324E92B630F454F,29,665000,1,,B,675000,10,1
105,CNG14,106,2014-01-30D08:50:20.867603600,50B9624540CD23ED,7,675000,1,,A,675000,161,1
106,CNG14,107,2014-01-30D08:50:29.121495300,7C22E701636347A3,3,675000,70,,A,674500,72,70
107,CNG14,108,2014-01-30D08:50:29.121495300,FA6ACA4D2D540C54,1,675000,-1,,B,674500,10,-1
108,CNG14,109,2014-01-30D08:50:29.121495300,FA6ACA4D2D540C54,1,674500,1,,B,674500,10,1
109,CNG14,110,2014-01-30D08:50:29.121495300,74224A0DC315A215,2,675000,-1,,B,674500,10,-1


In [179]:
temp_ask,temp_bid

(2, 14)

In [180]:
order_book_ask[0],order_book_bid[0]

(    Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  682000  0F50C53613C0F881                   1                    1
 1  683500  D8F8E619640F43AF                   1                    1
 2  689000  C4F76F586C344E54                 100                  100
 3  691000  6EA6E34859783E78                  10                   10
 4  699000  662E6B4847A22CA2                  10                   10
 5  700000  BFC92EA46AE44FA4                   1                    1
 6  717500  2BD5C16638561AB6                  10                   10,
     Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  665000  48714F4C6C254E45                  50                   50
 1  665000  C0F94F4C3F5B217B                  50                   50
 2  665000  4071CF4C6EE55105                  50                   50
 3  663500  FE234A199A1979B9                   1                    1
 4  663000  27586D726EF753F7                  10                   10
 5  660000  6CB1630

In [181]:
order_book_ask[1],order_book_bid[1]

(    Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  682000  0F50C53613C0F881                   1                    1
 1  683500  D8F8E619640F43AF                   1                    1
 2  689000  C4F76F586C344E54                 100                  100
 3  691000  6EA6E34859783E78                  10                   10
 4  699000  662E6B4847A22CA2                  10                   10
 5  700000  BFC92EA46AE44FA4                   1                    1
 6  717500  2BD5C16638561AB6                  10                   10
 7  725000  7A666A457D4F5C2F                   4                    4,
     Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  668000  2DD0417EFB04DFC4                   1                    1
 1  665000  48714F4C6C254E45                  50                   50
 2  665000  C0F94F4C3F5B217B                  50                   50
 3  665000  4071CF4C6EE55105                  50                   50
 4  663500  FE234A1

In [182]:
order_book_ask[2],order_book_bid[2]

(    Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  682000  0F50C53613C0F881                   1                    1
 1  682000  83A8692BEE72D0B2                   1                    1
 2  683500  D8F8E619640F43AF                   1                    1
 3  689000  C4F76F586C344E54                 100                  100
 4  691000  6EA6E34859783E78                  10                   10
 5  699000  662E6B4847A22CA2                  10                   10
 6  700000  BFC92EA46AE44FA4                   1                    1
 7  717500  2BD5C16638561AB6                  10                   10
 8  725000  7A666A457D4F5C2F                   4                    4,
     Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  668000  2DD0417EFB04DFC4                   1                    1
 1  665500  8142C93656273847                   1                    1
 2  665000  48714F4C6C254E45                  50                   50
 3  665000  C0F94F4

In [32]:
order_book_ask[temp_ask],order_book_bid[temp_bid]

(      Price       OrderNumber  QuantityDifference  QuantityDifference_
 0    668000  5872EF19009DE4DE                   5                   10
 1    668000  0B88CD76947A795A                  10                   10
 2    668000  23346D6BF357D597                   1                    1
 3    668000  2B114566DBD1C091                   1                    1
 4    668000  DEA1C3197F185E58                  10                   10
 5    668000  F63A621D1E32FDF3                  78                   78
 6    668000  F43AEA1DEA94C974                  20                   20
 7    668000  52268A8986FD677D                   8                    8
 8    668000  8DF0E563D3DFB61F                   8                    8
 9    668000  5AA60A81376A17EA                   8                    8
 10   668000  D03DE65D415B229B                   2                    2
 11   668000  F4320EC980AB612B                   8                    8
 12   668500  A382C93E4E1B331B                  45              

In [52]:
order_book_ask[0][['Price','QuantityDifference']][1:2].values.tolist()[0]

[683500, 1]

In [25]:
import csv
import sys

In [74]:
import csv
with open('order_book.txt', 'wb') as csvfile:
    f = csv.writer(csvfile)#, delimiter=' ',quotechar='|', quoting=csv.QUOTE_MINIMAL)

In [53]:
with open('eggs.txt','a') as f:
    spamwriter = csv.writer(f,delimiter=',',)
    spamwriter.writerow(order_book_ask[0][['Price','QuantityDifference']][0:1].values.tolist()[0])
    spamwriter.writerow(order_book_ask[0][['Price','QuantityDifference']][1:2].values.tolist()[0])
    spamwriter.writerow(order_book_ask[0][['Price','QuantityDifference']][2:3].values.tolist()[0])
    spamwriter.writerow(order_book_ask[0][['Price','QuantityDifference']][3:4].values.tolist()[0])
    spamwriter.writerow(order_book_ask[0][['Price','QuantityDifference']][4:5].values.tolist()[0])

In [96]:
with open('order_book.txt','a') as f:
    order_book = csv.writer(f)
    order_book.writerow([data.TimeStamp[i]])
    order_book = csv.writer(f,delimiter=',')
    for i in range(1,max(len(order_book_ask[0]),len(order_book_bid[0]))):
        order_book.writerow(a[i:i+1].values.tolist()[0])
        

In [81]:
data.TimeStamp[0]

'2014-01-30D04:27:08.581903900'

In [135]:
#a = pd.concat([order_book_bid[1][['Price','QuantityDifference']],order_book_ask[1][['Price','QuantityDifference']]],axis = 1)

In [134]:
#a

In [133]:
#pd.read_table('eggs.txt')#,sep = ',')

In [19]:
with open('order_book.csv', 'rb') as f:
    reader = csv.reader(f)
    for row in reader:
        print row

In [5]:
import time
start = time.time()
order_book_bid_merge = []
order_book_ask_merge = []
for i in range(0,len(order_book_bid)):
    a = order_book_bid[i][['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()#.sort(['Price'],ascending = [False])
    order_book_bid_merge.append(pd.DataFrame(a.values))
for i in range(0,len(order_book_ask)):
    a = order_book_ask[i][['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()#.sort(['Price'],ascending = [True])
    order_book_ask_merge.append(pd.DataFrame(a.values))
end = time.time()            
print ("Total time = %f"%(end - start)) 


Total time = 0.432648


In [17]:
order_book_bid_merge[7]

,0,1
0,660000,100
1,663000,10
2,663500,1
3,664000,1
4,665000,151
5,665500,1
6,666000,1
7,668000,1


In [10]:
order_book_ask_merge[0]

,0,1
0,682000,1
1,683500,1
2,689000,100
3,691000,10
4,699000,10
5,700000,1
6,717500,10


In [20]:
import time
start = time.time()
time_ = data.TimeStamp.unique()
time_bid = data[(data.BidOrAsk == 'B')].TimeStamp.unique()[1:]
time_ask = data[(data.BidOrAsk == 'A')].TimeStamp.unique()[1:]
bid_index = 0
ask_index = 0
a = 0
b = 0
first = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
first = pd.DataFrame(first.values)
time = pd.DataFrame([['TimeStamp',time_[0][11:29],'','']])
order_book_concat = time.append(first).reset_index(drop=True)
order_book = order_book_concat
#print order_book
time_series = 1
for time in time_[1:]:
    #print 'time' + time
    #print 'time_series = %d'%(time_series)
    time_to_concat = pd.DataFrame([['TimeStamp',time_[time_series][11:29],'','']])
    #print time_to_concat
    #print time_bid[a]
    #print time_ask[b]
    #print order_book_bid_merge[a]
    #print order_book_ask_merge[b]
    if time == time_bid[a] and time == time_ask[b]:
        #print '1 = ' + time
        bid_index = bid_index + 1
        ask_index = ask_index + 1
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        a = a + 1
        b = b + 1
    elif time == time_bid[a] and time != time_ask[b]: 
        #print '2 = ' +time
        bid_index = bid_index + 1        
        
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        a = a + 1
    elif time != time_bid[a] and time == time_ask[b]: 
        #print '3 = ' +time
        ask_index = ask_index + 1    
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        b = b + 1
    else:
        continue
    order_book = pd.concat([order_book,order_book_concat])    
    time_series = time_series + 1    
    
    #print order_book
end = time.time()            
print "Total time = %f"%(end - start) 


IndexError: list index out of range

In [31]:
order_book = pd.DataFrame(order_book.values)
order_book.to_csv('CNF14_01_03_order_book_final.csv',sep = '\t',encoding = 'utf-8')

In [30]:
import time
start = time.time()


time_ = data.TimeStamp.unique()
time_bid = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
time_ask = data[(data.BidOrAsk == 'A')].TimeStamp.unique()

bid_index = 0
ask_index = 0
a = 0
b = 0
first = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
first = pd.DataFrame(first.values)
time = pd.DataFrame([['TimeStamp',time_[0][11:29],'','']])
order_book_concat = time.append(first).reset_index(drop=True)
order_book = order_book_concat
#print order_book
a = 1
b = 1
time_series = 1
for time in time_[1:]:
    #print 'time' + time
    #print 'time_series = %d'%(time_series)
    time_to_concat = pd.DataFrame([['TimeStamp',time_[time_series][11:29],'','']])
    #print time_to_concat
    #print time_bid[a-1]
    #print time_ask[b-1]    
    #print time_bid[a]
    #print time_ask[b]
    #print order_book_bid_merge[a]
    #print order_book_ask_merge[b]
    if time == time_bid[a] and time == time_ask[b]:
        #print '1 = ' + time
        #a = a + 1
        #b = b + 1
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[a].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[b].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        a = a + 1
        b = b + 1
    elif time == time_bid[a] and time != time_ask[b]: 
        #print '2 = ' +time
        ##a = a + 1
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[a].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[b-1].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        a = a + 1
        
    elif time != time_bid[a] and time == time_ask[b]: 
        #print '3 = ' +time
        ##b = b + 1
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[a-1].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[b].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        b = b + 1
    else:
        continue
    order_book = pd.concat([order_book,order_book_concat])    
    time_series = time_series + 1    
    #print order_book
end = time.time()            
print "Total time = %f"%(end - start) 


IndexError: index 45578 is out of bounds for axis 0 with size 45578

In [7]:
order_book = pd.DataFrame(order_book.values)
order_book.to_csv('CNF14_01_13_order_book_final.csv',sep = '\t',encoding = 'utf-8')